In [86]:
import cv2
import matplotlib.pyplot as plt
from glob import glob
import numpy as np
import imageio
from skimage import img_as_float32
from natsort import natsorted

path_pairs = list(zip(
    natsorted(glob('../puzzle_corners_1024x768/images-1024x768/*.png')),
    natsorted(glob('../puzzle_corners_1024x768/masks-1024x768/*.png')),
))

imgs = np.array([img_as_float32(imageio.imread(ipath)) for ipath, _ in path_pairs])
msks = np.array([img_as_float32(imageio.imread(mpath)) for _, mpath in path_pairs])

imgs_flat = imgs.reshape(imgs.shape[0],-1)
msks_flat = msks.reshape(msks.shape[0],-1)


In [87]:
from sklearn.model_selection import train_test_split
imgs_train, imgs_test_val, msks_train, msks_test_val = train_test_split(
    imgs_flat, msks_flat, test_size=0.14, random_state=42)

In [88]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=6)
kf.get_n_splits(imgs_train)

6

In [89]:
from scipy.stats import multivariate_normal

In [90]:
multivariate_normal.pdf(10, mean=2.5, cov=0.5)

2.1006826890574786e-25

In [91]:
imgs_train.shape

(41, 2359296)

In [95]:
k = 4
num_steps = 1000
I = imgs_train.shape[0]
mean = np.random.random(I)
cov = np.random.random((I,I))

cov += cov.T
cov += I*np.eye(I)
cov*=10

r = np.zeros((I, k))

for i in range(num_steps):
    multivariate_normal.pdf(imgs_train, mean=mean, cov=cov)
    #r = 

ValueError: operands could not be broadcast together with shapes (41,2359296) (41,) 

In [84]:
cov

array([[429.02381991,   9.51489884,  12.09680168, ...,   5.89979628,
         13.2006359 ,  11.58900693],
       [  9.51489884, 410.6641184 ,  14.91403445, ...,   6.20425396,
          2.4910897 ,  13.57550745],
       [ 12.09680168,  14.91403445, 420.30140028, ...,  11.26455811,
         12.286122  ,  13.32919377],
       ...,
       [  5.89979628,   6.20425396,  11.26455811, ..., 415.02022956,
         15.42569435,  10.46231126],
       [ 13.2006359 ,   2.4910897 ,  12.286122  , ...,  15.42569435,
        428.15135297,   3.37025623],
       [ 11.58900693,  13.57550745,  13.32919377, ...,  10.46231126,
          3.37025623, 429.25753169]])

In [85]:
r

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
